<a href="https://colab.research.google.com/github/davidlealo/vocacional-test/blob/main/notebooks/03_vocacional_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Asistente Vocacional FPUC — Colab

Este notebook implementa un **asistente vocacional** usando **Azure OpenAI** con:
- Embeddings (`emb-vocatest`) para búsqueda semántica sobre *Binder1.pdf*
- Chat (`phi4mini-chat`) para respuestas contextuales
- Variables seguras desde `.env` con *fallback* a `getpass()`

> **Requisitos previos**  
> 1) Tener un recurso Azure OpenAI con deployments: `phi4mini-chat` y `emb-vocatest`  
> 2) Subir tu archivo `Binder1.pdf` a la carpeta `data/` de este entorno  
> 3) (Opcional) Subir `.env` en la raíz del proyecto con tus credenciales


In [ ]:
# ⬇️ Instalación de dependencias
!pip -q install pdfplumber python-dotenv requests tqdm scikit-learn pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 89.2 MB/s eta 0:00:00


In [ ]:
# 🔐 Carga de variables de entorno desde .env y fallback interactivo
import os
from getpass import getpass

try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception as e:
    print("python-dotenv no disponible, se usará solo input interactivo.")

if not os.getenv("AZURE_OPENAI_KEY"):
    os.environ["AZURE_OPENAI_KEY"] = getpass("Introduce AZURE_OPENAI_KEY: ")

if not os.getenv("AZURE_OPENAI_ENDPOINT"):
    os.environ["AZURE_OPENAI_ENDPOINT"] = input("Introduce AZURE_OPENAI_ENDPOINT (ej: https://oai-vocatest.services.ai.azure.com/): ").strip()

if not os.getenv("AZURE_OPENAI_DEPLOYMENT"):
    os.environ["AZURE_OPENAI_DEPLOYMENT"] = input("Introduce AZURE_OPENAI_DEPLOYMENT (ej: phi4mini-chat): ").strip()

if not os.getenv("AZURE_OPENAI_EMBEDDING"):
    os.environ["AZURE_OPENAI_EMBEDDING"] = input("Introduce AZURE_OPENAI_EMBEDDING (ej: emb-vocatest): ").strip()

print("✅ Variables cargadas.")

✅ Variables cargadas.


In [ ]:
# 📚 Importaciones y helpers
import pdfplumber
import re
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

API_KEY = os.environ["AZURE_OPENAI_KEY"]
ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
CHAT_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT"]
EMB_DEPLOYMENT  = os.environ["AZURE_OPENAI_EMBEDDING"]

CHAT_URL = f"{ENDPOINT}openai/deployments/{CHAT_DEPLOYMENT}/chat/completions?api-version=2024-05-01-preview"
EMB_URL  = f"{ENDPOINT}openai/deployments/{EMB_DEPLOYMENT}/embeddings?api-version=2024-05-01-preview"

HEADERS = {"Content-Type": "application/json", "api-key": API_KEY}

def clean_text(t: str) -> str:
    t = re.sub(r"[\u0000-\u001F]+", " ", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

def chunk_text(text: str, max_chars: int = 5000, overlap: int = 500):
    # max_chars ~ aprox 800-1000 tokens según idioma; ajustable
    chunks = []
    i = 0
    while i < len(text):
        chunk = text[i:i+max_chars]
        chunks.append(chunk.strip())
        i += max_chars - overlap
    return [c for c in chunks if c]

In [ ]:
# 📄 Extraer texto desde Binder1.pdf (sube tu archivo a ./data/Binder1.pdf)
from pathlib import Path

pdf_path = Path("data/Binder1.pdf")
assert pdf_path.exists(), "❌ No se encontró data/Binder1.pdf. Sube el archivo a la carpeta data/."

pages_text = []
with pdf_path.open("rb") as f:
    with pdfplumber.open(f) as pdf:
        for page in pdf.pages:
            txt = page.extract_text() or ""
            pages_text.append(txt)

full_text = clean_text("\n".join(pages_text))
print("✅ Texto extraído. Longitud de caracteres:", len(full_text))

# Dividir en chunks
chunks = chunk_text(full_text, max_chars=5000, overlap=500)
print(f"✅ Chunks generados: {len(chunks)}")

✅ Texto extraído. Longitud de caracteres: 62401
✅ Chunks generados: 14


In [ ]:
# 🧠 Generar embeddings de todos los chunks (puede tardar unos minutos según largo)
def get_embeddings(text_list):
    resp = requests.post(EMB_URL, headers=HEADERS, data=json.dumps({"input": text_list}))
    if resp.status_code != 200:
        raise RuntimeError(f"Error embeddings: {resp.status_code} {resp.text}")
    data = resp.json()["data"]
    return [d["embedding"] for d in data]

# Procesar por lotes para evitar payloads grandes
batch_size = 16
all_embeddings = []
for i in tqdm(range(0, len(chunks), batch_size), desc="Embedding batches"):
    batch = chunks[i:i+batch_size]
    embs = get_embeddings(batch)
    all_embeddings.extend(embs)

emb_arr = np.array(all_embeddings, dtype=np.float32)
print("✅ Matriz de embeddings:", emb_arr.shape)

# Guardar a disco
out_df = pd.DataFrame({"chunk": chunks, "embedding": list(all_embeddings)})
out_df.to_pickle("binder_embeddings.pkl")
print("💾 Guardado: binder_embeddings.pkl")

Embedding batches: 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]

✅ Matriz de embeddings: (14, 1536)
💾 Guardado: binder_embeddings.pkl


In [ ]:
# 🔍 Búsqueda semántica (top-k)
def retrieve_context(query: str, top_k: int = 3):
    r = requests.post(EMB_URL, headers=HEADERS, data=json.dumps({"input": query}))
    if r.status_code != 200:
        raise RuntimeError(f"Error embedding query: {r.status_code} {r.text}")
    q_emb = np.array(r.json()["data"][0]["embedding"], dtype=np.float32).reshape(1, -1)
    db = pd.read_pickle("binder_embeddings.pkl")
    M = np.vstack(db["embedding"].values)
    sims = cosine_similarity(q_emb, M)[0]
    idxs = sims.argsort()[::-1][:top_k]
    hits = db.iloc[idxs]
    # devolver texto concatenado
    context = "\n\n---\n\n".join(hits["chunk"].tolist())
    return context, hits[["chunk"]]

# Pequeña prueba de recuperación
ctx, refs = retrieve_context("¿Qué es la gratuidad y cómo postular?")
print(ctx[:500], "...")

necer al 60% de menores ingresos. Gratuidad Requisito institucional: Instituciones acreditadas y adscritas a la gratuidad. Es importante que sepas que para obtener la gratuidad no se necesita ningún requisito académico, aunque sí serán requeridos para poder matricularse en las distintas institu- ciones. Beneficio destinado a estudiantes egresados de enseñanza media con rendimiento académico meritorio. Financia el arancel referencial de tu carrera. Beca Requisito económico: Pertenecer al 70% de l ...


In [ ]:
# 🧾 Carga automática del prompt institucional desde archivo (local o Colab)
from pathlib import Path

# Detecta si estás en Colab o en entorno local
try:
    import google.colab  # noqa
    running_in_colab = True
except ImportError:
    running_in_colab = False

# Posibles rutas del archivo
possible_paths = [
    Path("prompt_vocacional.txt"),                  # misma carpeta del notebook (Colab)
    Path("/content/prompt_vocacional.txt"),         # ruta absoluta común en Colab
    Path("../prompt_vocacional.txt"),               # raíz del proyecto local (VS Code)
    Path("../../prompt_vocacional.txt"),            # un nivel más arriba
]

# Busca el primer archivo existente
prompt_file = next((p for p in possible_paths if p.exists()), None)

if prompt_file:
    with open(prompt_file, "r", encoding="utf-8") as f:
        PROMPT_SISTEMA = f.read()
    print(f"✅ Prompt cargado desde: {prompt_file}")
else:
    print("⚠️ No se encontró 'prompt_vocacional.txt'. Se usará versión reducida por defecto.")
    PROMPT_SISTEMA = """
    Este asistente está diseñado para ayudar a postulantes a carreras universitarias o técnicas en Chile
    a tomar decisiones informadas sobre su futuro académico y profesional. Trabaja en colaboración con la
    Fundación Por Una Carrera y la ONG Innovacien. Proporciona orientación sobre admisión, becas, beneficios
    y vida universitaria de manera inclusiva y clara.
    """

print(PROMPT_SISTEMA[:250] + "...\n")  # muestra una vista previa


✅ Prompt cargado desde: prompt_vocacional.txt
Este asistente está diseñado para ayudar a postulantes a carreras universitarias o técnicas en Chile a tomar decisiones informadas sobre su futuro académico y profesional. Trabaja en colaboración con la Fundación Por Una Carrera y la ONG Innovacien, ...



In [ ]:
# 🧾 Prompt institucional (puedes editarlo o cargarlo desde prompt_vocacional.txt)
PROMPT_SISTEMA = """Este asistente está diseñado para ayudar a postulantes a carreras universitarias o técnicas en Chile a tomar decisiones informadas sobre su futuro académico y profesional. Trabaja en colaboración con la Fundación Por Una Carrera y la ONG Innovacien, por lo que su enfoque es inclusivo, confiable, claro y siempre centrado en el bienestar del estudiante.

Proporciona orientación sobre procesos de admisión, requisitos de postulación, becas y beneficios estatales, opciones académicas, vida universitaria y decisiones vocacionales.

El asistente incorpora preguntas frecuentes y actividades de autoconocimiento extraídas del material educativo de Fundación Por Una Carrera (Binder1.pdf) y recursos oficiales del DEMRE/FUAS. Debe citar o recomendar recursos cuando corresponda.

El asistente debe:
- Explicar con claridad y sin tecnicismos innecesarios.
- Adaptar sus respuestas a la realidad chilena actual.
- Mostrar sensibilidad frente a contextos de vulnerabilidad social y educativa.
- Proporcionar enlaces, fuentes oficiales y recursos cuando estén disponibles.
- Promover la toma de decisiones informadas sin imponer opciones.

Debe evitar:
- Reemplazar la orientación profesional presencial.
- Dar consejos cerrados o categóricos que limiten opciones.
- Suponer información sin fundamentos o sin contexto.

Al comenzar, saluda indicando que esta es una iniciativa conjunta entre la Fundación Por Una Carrera (https://www.instagram.com/porunacarrera) e Innovacien (https://www.instagram.com/innovacien), e invita a seguir ambas cuentas en Instagram y a compartir su experiencia con el chat.
"""

In [ ]:
# 💬 Función de respuesta con contexto
def chat_with_context(user_query: str, top_k: int = 3, temperature: float = 0.4, max_tokens: int = 600):
    context, refs = retrieve_context(user_query, top_k=top_k)
    payload = {
        "messages": [
            {"role": "system", "content": PROMPT_SISTEMA},
            {"role": "user", "content": f"Contexto relevante:\n{context}\n\nPregunta del usuario: {user_query}"},
        ],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    resp = requests.post(CHAT_URL, headers=HEADERS, data=json.dumps(payload))
    if resp.status_code != 200:
        raise RuntimeError(f"Error chat: {resp.status_code} {resp.text}")
    answer = resp.json()["choices"][0]["message"]["content"]
    return answer, refs

# Prueba rápida
ans, refs = chat_with_context("¿Cómo postulo a beneficios estudiantiles del Estado?")
print("🤖 Respuesta:\n", ans[:800], "...")

🤖 Respuesta:
 Para postular a beneficios estudiantiles del Estado en Chile, generalmente debes seguir estos pasos:

1. **Identifica los beneficios a los que estás calificado**: Revisa la información disponible sobre los diferentes beneficios estatales, como la Gratuidad, Becas de la Bicentenario, Becas Vocacionales, Becas de Arancel, y otros. Cada uno tiene requisitos específicos en cuanto a nivel socioeconómico, rendimiento académico y institución educativa.

2. **Consulta el formulario de solicitud**: El Ministerio de Educación proporciona un formulario único de acreditación socioeconómica (FUA) para la postulación a beneficios estatales. Este formulario te permite declarar tu situación socioeconómica y académica, y es el primer paso para acceder a los beneficios.

3. **Reúne la documentación necesari ...


In [ ]:
# 🧑‍💻 Interfaz simple de conversación (ejecuta esta celda y luego llama a chat_with_context)
q = "Quiero estudiar pedagogía pero no sé si califico para gratuidad. ¿Qué debo revisar?"
ans, refs = chat_with_context(q, top_k=3)
print("🧍 Usuario:", q)
print("\n🤖 Asistente:\n", ans)
print("\n🔎 Fragmentos usados (primeros 200 chars de cada uno):")
for i, ch in enumerate(refs["chunk"].tolist(), 1):
    print(f"[{i}] {ch[:200]}...")

🧍 Usuario: Quiero estudiar pedagogía pero no sé si califico para gratuidad. ¿Qué debo revisar?

🤖 Asistente:
 Para determinar si calificas para la gratuidad en una carrera de pedagogía en Chile, necesitas revisar los siguientes requisitos:

1. Requisito Institucional: La institución donde deseas estudiar debe estar acreditada. Puedes verificar la acreditación de la institución visitando el sitio web del Ministerio de Educación o consultando el Sistema Nacional de Acreditación (SINA).

2. Requisito Económico: Debes pertenecer al 60% de menores ingresos del país. Para comprobar tu situación económica, puedes solicitar una evaluación del Ministerio de Educación. Ellos te ayudarán a determinar si calificas para la gratuidad basada en tu situación familiar.

3. Requisito Académico: Para la gratuidad, no hay un requisito académico específico, pero es importante tener en cuenta que tu puntaje en las pruebas obligatorias (Competencia Lectora y Competencia Matemática) puede influir en tu elegib

In [ ]:
# 💬 Realizar una pregunta al asistente vocacional y mostrar respuesta
import os
import json
import requests
from dotenv import load_dotenv

# === CARGAR VARIABLES DE ENTORNO ===
load_dotenv()

API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")

if not all([API_KEY, ENDPOINT, DEPLOYMENT]):
    raise ValueError("❌ Faltan variables de entorno: revisa tu archivo .env o define las variables manualmente.")

# === CONSTRUIR ENDPOINT DE CHAT ===
CHAT_URL = f"{ENDPOINT}openai/deployments/{DEPLOYMENT}/chat/completions?api-version=2024-05-01-preview"

# === CABECERAS ===
headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY
}

# === HACER UNA PREGUNTA ===
pregunta_usuario = input("🎓 Ingresa tu pregunta vocacional: ")

data = {
    "messages": [
        {"role": "system", "content": PROMPT_SISTEMA},
        {"role": "user", "content": pregunta_usuario}
    ],
    "temperature": 0.7,
    "max_tokens": 500
}

# === SOLICITUD A AZURE ===
response = requests.post(CHAT_URL, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    respuesta = response.json()["choices"][0]["message"]["content"]
    print("\n🧭 Respuesta del asistente:\n")
    print(respuesta)
else:
    print("❌ Error:", response.status_code)
    print(response.text)


🎓 Ingresa tu pregunta vocacional: Estoy en mi segunda carrera y quiero seguir estudiando con gratuidad

🧭 Respuesta del asistente:

Es fantástico saber que estás adelante con tu educación y buscas continuar con un grado en gratuidad. Aquí te dejo algunos consejos y recursos que podrían ayudarte a seguir tu camino en la gratuidad:

1. **Investiga Instituciones Gratuitas**: Existen muchas universidades y universidades públicas en México que ofrecen programas de grado gratis. Algunos de estos son la Universidad Nacional Autónoma de México (UNAM), la Universidad de Guadalajara, la Universidad Autónoma de Nuevo León, entre otras. Revisa las páginas oficiales de estas instituciones para encontrar programas que se ajusten a tus intereses.

2. **Verifica los Requisitos**: Cada institución tiene sus propios requisitos para ingresar a programas de grado gratuito. Esto puede incluir haber cursado una carrera preparatoria, tener calificaciones específicas, no haber cursado una carrera de posgrado,